This sets up to components that we need for the program

In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt
from astropy.io import fits
%matplotlib inline
from matplotlib import rcParams

rcParams['figure.figsize'] = [10., 8.]

 This finds the file in my documents and renames it to fname

In [ ]:
fname = fits.util.get_testdata_filepath('image.fits')
hdul = fits.open(fname)
hdul.info()

This helps read the file and stores it in "data"

In [ ]:
data = fits.getdata(fname)
print(type(data))
print(data.shape)

plots the image of fits image

In [ ]:
plt.imshow(data, cmap='gray')
plt.colorbar()

gets the data of the background

In [ ]:
bkg = sep.Background(data)

prints the background color data numbers

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

stores the image of the background to bkg_image

In [ ]:
bkg_image = bkg.back()

plts bkg.back into an image

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

stores the other color found in the background in bkg_rms

In [ ]:
bkg_rms = bkg.rms()

plots the bkg_rms image 

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

this the subtraction of the background from the original image

In [ ]:
data_sub = data - bkg

this helps detect the objects that are found in the picture

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

Prints out the number of objects found in the picture

In [ ]:
len(objects)

This coordinates the center of the objects and detects where they are.
This also circles the objects in red using Ellipse

In [ ]:

from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

Name of available fields

In [ ]:
objects.dtype.names

This performs circular aperture photometry with 3 pixel radius at the location of the objects

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

This shows the result of the first 10 objects found

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

Saves the figures to png files

In [ ]:
plt.imsave("figure1.png", data)
plt.imsave("figure2.png", bkg_image)
plt.imsave("figure3.png", bkg_rms)
plt.imsave("figure4.png", data_sub)